In [2]:
!pip3 install geopy

     |████████████████████████████████| 111 kB 6.2 MB/s eta 0:00:01


In [3]:
!pip install geopy

  Using cached https://files.pythonhosted.org/packages/07/e1/9c72de674d5c2b8fcb0738a5ceeb5424941fefa080bfe4e240d0bacb5a38/geopy-2.0.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/8b/62/26ec95a98ba64299163199e95ad1b0e34ad3f4e176e221c40245f211e425/geographiclib-1.50-py3-none-any.whl


In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [5]:
!pip3 install lxml
!pip install lxml
!pip3 install BeautifulSoup4
!pip install BeautifulSoup4
!pip3 install html5lib
!pip install html5lib

     |████████████████████████████████| 5.4 MB 7.7 MB/s eta 0:00:01
     |████████████████████████████████| 5.5MB 5.6MB/s eta 0:00:01
     |████████████████████████████████| 115 kB 11.1 MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/d1/41/e6495bd7d3781cee623ce23ea6ac73282a373088fcd0ddc809a047b18eae/beautifulsoup4-4.9.3-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/02/fb/1c65691a9aeb7bd6ac2aa505b84cb8b49ac29c976411c6ab3659425e045f/soupsieve-2.1-py3-none-any.whl
     |████████████████████████████████| 112 kB 10.1 MB/s eta 0:00:01


In [6]:
dict = {'Postcode':['EC1', 'EC2', 'EC3', 'WC1','WC2'], 
        'Areaname':['Finsbury','Bishopsgate','Fenchurch Street','Mount Pleasant','Strand'],
        'Latitude':[51.5,51.5225,51.5084,51.5236,51.5131],
        'Longitude':[-0.0833,-0.0857,-0.1255,-0.1122,-0.1221]
       } 
London_postcodes = pd.DataFrame(dict)
display(London_postcodes)

,Postcode,Areaname,Latitude,Longitude
0,EC1,Finsbury,51.5000,-0.0833
1,EC2,Bishopsgate,51.5225,-0.0857
2,EC3,Fenchurch Street,51.5084,-0.1255
3,WC1,Mount Pleasant,51.5236,-0.1122
4,WC2,Strand,51.5131,-0.1221


In [25]:
address = 'London, UK'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London are 51.5073219, -0.1276474.


In [8]:
# create map of New York using latitude and longitude values
london_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, areaname in zip(London_postcodes['Latitude'], London_postcodes['Longitude'], London_postcodes['Areaname']):
    label = '{}'.format(areaname)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(london_map)  
    
london_map

In [9]:
CLIENT_ID = 'U5XJ0FHZBFD25EMEQHMAJY5WYPQKDUB4LCXOIQFC3AHKRY3A' # your Foursquare ID
CLIENT_SECRET = 'HNBTZGRCABTN1BIZYMJULRA02EY4BORHFCM0MVLJIA3WM0K4' # your Foursquare Secret
VERSION = '20201215' # Foursquare API version
latitude = '51.5'
longitude = '-0.0833'
RADIUS = 500
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: U5XJ0FHZBFD25EMEQHMAJY5WYPQKDUB4LCXOIQFC3AHKRY3A
CLIENT_SECRET:HNBTZGRCABTN1BIZYMJULRA02EY4BORHFCM0MVLJIA3WM0K4


In [10]:
#url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
#            "U5XJ0FHZBFD25EMEQHMAJY5WYPQKDUB4LCXOIQFC3AHKRY3A", 
#            "HNBTZGRCABTN1BIZYMJULRA02EY4BORHFCM0MVLJIA3WM0K4", 
#            "20201208", 
#            "40.7896239", 
#            "-73.9598939", 
#            "500", 
#            "100")
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    500, 
    100)
results = requests.get(url).json()
#results

In [11]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [12]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.location.postalCode','venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

#nearby_venues.head(101)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
def getNearbyVenues(code, name,latitudes, longitudes):
    i=0
    venues_list=[]
    for lat, lng in zip(latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            '500', 
            '100')
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            code[i],
            name[i],
            #v['venue']['location']['postalCode'], 
            v['venue']['name'], 
            v['venue']['categories'][0]['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng']) for v in results])
        i=i+1
    nearby_venues_all = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues_all.columns = ['mainpostalcode', 
                  'mainareaname', 
                  #'venuepostalcode', 
                  'venuename', 
                  'venuecategory', 
                  'venuelat', 
                  'venuelng']
    
    return(nearby_venues_all)

In [14]:
london_venues_all = getNearbyVenues(London_postcodes["Postcode"], London_postcodes["Areaname"], London_postcodes["Latitude"], London_postcodes["Longitude"])
london_venues_all.head()

,mainpostalcode,mainareaname,venuename,venuecategory,venuelat,venuelng
0,EC1,Finsbury,Franco Manca,Pizza Place,51.500175,-0.081578
1,EC1,Finsbury,White Cube,Art Gallery,51.499643,-0.081496
2,EC1,Finsbury,Leathermarket Gardens,Garden,51.500655,-0.083739
3,EC1,Finsbury,Comptoir Gourmand,Bakery,51.500847,-0.081855
4,EC1,Finsbury,Fine Foods,Deli / Bodega,51.498385,-0.084026


In [15]:
london_venues_all_temp = london_venues_all
# one hot encoding
london_onehot = pd.get_dummies(london_venues_all_temp[['venuecategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
london_onehot['venuename'] = london_venues_all_temp['venuename'] 

# move neighborhood column to the first column
fixed_columns = [london_onehot.columns[-1]] + list(london_onehot.columns[:-1])
london_onehot = london_onehot[fixed_columns]

london_onehot.shape

(430, 123)

In [16]:
london_grouped = london_onehot.groupby('venuename').mean().astype(float).reset_index()
london_grouped.head(20)

,venuename,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bar,Beer Bar,Beer Garden,Bike Shop,Bookstore,Boutique,Boxing Gym,Brasserie,Breakfast Spot,Building,Burger Joint,Burrito Place,Café,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Cosmetics Shop,Costume Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Donut Shop,Electronics Store,English Restaurant,Event Space,Falafel Restaurant,Fast Food Restaurant,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,Gay Bar,Gelato Shop,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,History Museum,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Lebanese Restaurant,Market,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Monument / Landmark,Museum,Neighborhood,New American Restaurant,Noodle House,North Indian Restaurant,Opera House,Outdoor Sculpture,Pakistani Restaurant,Park,Pedestrian Plaza,Peruvian Restaurant,Pharmacy,Piano Bar,Pizza Place,Plaza,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Plaza,Social Club,Soup Place,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sports Bar,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Theater,Theme Park Ride / Attraction,Toy / Game Store,Turkish Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,% Arabica,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,& Other Stories,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1Rebel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,214 Bermondsey,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5cc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [17]:
for venue in london_grouped['venuename']:
    temp = london_grouped[london_grouped['venuename'] == venue].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    #print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    #print('\n')

In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['venuename']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
london_venues_sorted = pd.DataFrame(columns=columns)
london_venues_sorted['venuename'] = london_grouped['venuename']

for ind in np.arange(london_grouped.shape[0]):
    london_venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

london_venues_sorted.head(20)

,venuename,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,% Arabica,Coffee Shop,Yoga Studio,Fountain,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Donut Shop,Electronics Store,English Restaurant
1,& Other Stories,Women's Store,Fountain,Costume Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Donut Shop,Electronics Store,English Restaurant
2,1Rebel,Boxing Gym,French Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Donut Shop,Electronics Store,English Restaurant,Event Space
3,214 Bermondsey,Cocktail Bar,Yoga Studio,Fountain,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Donut Shop,Electronics Store,English Restaurant
4,5cc,Cocktail Bar,Yoga Studio,Fountain,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Donut Shop,Electronics Store,English Restaurant
5,AMT Coffee,Coffee Shop,Yoga Studio,Fountain,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Donut Shop,Electronics Store,English Restaurant
6,Abuelo,Coffee Shop,Yoga Studio,Fountain,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Donut Shop,Electronics Store,English Restaurant
7,Admiralty Arch,Monument / Landmark,Yoga Studio,Fountain,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Donut Shop,Electronics Store,English Restaurant
8,Aesop,Cosmetics Shop,Yoga Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Donut Shop,Electronics Store,English Restaurant,Event Space
9,Aldwych Theatre,Theater,Yoga Studio,Fountain,Costume Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Donut Shop,Electronics Store


In [20]:
# set number of clusters
kclusters = 5

london_grouped_clustering = london_grouped.drop('venuename', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#london_venues_all.head()

In [21]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

london_merged = london_venues_all


# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
london_merged = london_merged.join(london_venues_sorted.set_index("venuename"), how='inner', on='venuename') #neighborhoods_venues_sorted

#neighborhoods_venues_sorted.head(1000)
#manhattan_data.dtypes
#manhattan_merged1.dtypes
#neighborhoods_venues_sorted.dtypes
#neighborhoods_venues_sorted.head(1000)
london_merged.head(100) # check the last columns!
#manhattan_merged.dtypes

,mainpostalcode,mainareaname,venuename,venuecategory,venuelat,venuelng,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,EC1,Finsbury,Franco Manca,Pizza Place,51.500175,-0.081578,Pizza Place,Yoga Studio,Cosmetics Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Donut Shop,Electronics Store,English Restaurant
104,EC2,Bishopsgate,Franco Manca,Pizza Place,51.518780,-0.083448,Pizza Place,Yoga Studio,Cosmetics Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Donut Shop,Electronics Store,English Restaurant
184,EC3,Fenchurch Street,Franco Manca,Pizza Place,51.510909,-0.122780,Pizza Place,Yoga Studio,Cosmetics Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Donut Shop,Electronics Store,English Restaurant
354,WC2,Strand,Franco Manca,Pizza Place,51.510909,-0.122780,Pizza Place,Yoga Studio,Cosmetics Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Donut Shop,Electronics Store,English Restaurant
1,EC1,Finsbury,White Cube,Art Gallery,51.499643,-0.081496,Art Gallery,Yoga Studio,French Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Donut Shop,Electronics Store,English Restaurant
2,EC1,Finsbury,Leathermarket Gardens,Garden,51.500655,-0.083739,Garden,Yoga Studio,Fountain,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Donut Shop,Electronics Store,English Restaurant
3,EC1,Finsbury,Comptoir Gourmand,Bakery,51.500847,-0.081855,Bakery,Yoga Studio,French Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Donut Shop,Electronics Store,English Restaurant
4,EC1,Finsbury,Fine Foods,Deli / Bodega,51.498385,-0.084026,Deli / Bodega,Yoga Studio,Cosmetics Shop,Dance Studio,Department Store,Dessert Shop,Donut Shop,Electronics Store,English Restaurant,Event Space
5,EC1,Finsbury,Casse-Crôute,French Restaurant,51.500457,-0.081704,French Restaurant,Cosmetics Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Donut Shop,Electronics Store,English Restaurant,Event Space
6,EC1,Finsbury,F*ckoffee,Coffee Shop,51.499358,-0.081282,Coffee Shop,Yoga Studio,Fountain,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Donut Shop,Electronics Store,English Restaurant


In [26]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

def colour(i):
        switcher={
                'EC1':0,
                'EC2':1,
                'EC3':2,
                'WC1':3,
                'WC2':4,
                }
        return switcher.get(i,0)

clr = ['red','green','blue','white','black']

for venuename, mainpostalcode, lat, lon in zip(london_venues_all['venuename'], london_venues_all['mainpostalcode'], london_venues_all['venuelat'], london_venues_all['venuelng']):
    label = folium.Popup(str(venuename), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=clr[colour(mainpostalcode)],
        fill=True,
        fill_color=clr[colour(mainpostalcode)],
        fill_opacity=0.8).add_to(map_clusters)
       
map_clusters

In [24]:
a = london_merged.loc[0, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]
a.head()

mainareaname                   Finsbury
venuelng                     -0.0815784
1st Most Common Venue       Pizza Place
2nd Most Common Venue       Yoga Studio
3rd Most Common Venue    Cosmetics Shop
Name: 0, dtype: object

In [ ]:
a = london_merged.loc[1, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]
a.head()

In [ ]:
a = london_merged.loc[2, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]
a.head()

In [ ]:
a = london_merged.loc[3, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]
a.head()

In [ ]:
a = london_merged.loc[4, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]
a.head()

In [ ]:
a = london_merged.loc[5, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]
a.head()